![SGSSS Logo](../img/SGSSS_Stacked.png)

# Understanding and Modelling Trajectories in the Social Sciences

Wednesday 25 June 2024

Dr Diarmuid McDonnell

## Inequalities in charity density across UK local authorities: 1971-2021

This notebook replicates elements of a paper examining local authority-level trajectories in the numbers of charities per capita 
over time.

```
Abstract

There are longstanding concerns about unevenness in the distribution of voluntary organisations across local areas, re-emphasised by the sector’s role in responding to the pandemic. Using longitudinal data on a subset of voluntary organisations across six census periods (1971-2021), we estimate growth trajectories in the number of charities per capita for local authorities across England and Wales. We assess whether the shape of these trajectories is associated with an area’s demographic and geographic characteristics, including its level of material deprivation, ethnic composition and regional location. We reveal sizeable, enduring and widening differences between local authorities over time, but find a homogenous growth trajectory for all areas. Our findings suggest that national efforts are needed to address inequalities in the distribution of charitable organisations, as well as reverse recent declines in the per capita density of these organisations. 
```

### Aims

This lesson has two aims:
1. Demonstrate how to estimate and interpret growth-curve models to answer social science research questions.
2. Cultivate your computational skills through the use of the statistical programming langauge *R*. For example, there are a number of opportunities for you to amend or write R syntax (code).

### Guide to using this resource

This learning resource was built using <a href="https://jupyter.org/" target=_blank>Jupyter Notebook</a>, an open-source software application that allows you to mix code, results and narrative in a single document. As <a href="https://jupyter4edu.github.io/jupyter-edu-book/" target=_blank>Barba et al. (2019)</a> espouse:
> In a world where every subject matter can have a data-supported treatment, where computational devices are omnipresent and pervasive, the union of natural language and computation creates compelling communication and learning opportunities.

If you are familiar with Jupyter notebooks then skip ahead to the main content (*Quick Recap: Analysing Data*). Otherwise, the following is a quick guide to navigating and interacting with the notebook.

### Interaction

**You only need to execute the code that is contained in sections which are marked by `[]`.**

To execute a cell, click or double-click the cell and press the `Play` button next to the cell or select the `Run` button on the top toolbar (*Runtime > Run the focused cell*); you can also use the keyboard shortcuts `Shift + Enter` or `Ctrl + Enter`).

Try it for yourself:

In [ ]:
name <- readline(prompt="Enter name: ")
print(paste("Hi,", name, "enjoy learning more about R and statistical models!"))

Notebooks are sequential, meaning code should be executed in order (top to bottom). For example, the following code won't work:

In [ ]:
x * 5

As the error message suggests, there is no object (variable) called `x`, therefore we cannot do any calculations with it. 

Let's try a sequential approach:

In [ ]:
x <- 10 # create an object called 'x' and give it the value '10'

In [ ]:
x * 5 # multiply 'x' by 5

### Learn more

Jupyter notebooks provide rich, flexible features for conducting and documenting your data analysis workflow. To learn more about additional notebook features, we recommend working through some of the <a href="https://github.com/darribas/gds19/blob/master/content/labs/lab_00.ipynb" target=_blank>materials</a> provided by Dani Arribas-Bel at the University of Liverpool. 

### Learner input

Throughout the lessons there times when you need to do the following activities:
* **TASK:** A coding task for you to complete (e.g. analyse different variables).
* **QUESTION:** A question regarding your interpretation of some code or a technique (e.g. what is the piece of code doing?).
* **EXERCISE:** A data analysis challenge for you to complete.

## Estimating growth-curve models

We will employ a sequential process for estimating these types of models (Leckie and Browne, 2023):
1. Examine data structure and descriptives
2. Estimate a random intercept (null) model
3. Estimate a random coefficent / slope model
4. Estimate a random coefficient / slope model with time-invariant covariates
5. Estimate a random coefficient / slope model with time-invariant and time-varying covariates
6. Use final model to generate estimates of the random intercepts and coefficients

### Install and load packages

Note you only have to install the packages **once** per machine, but you have to load them (`library(package)`) every time you start a new R session.

In [ ]:
install.packages("aod")
install.packages("tidyverse")
install.packages("ggplot2")
install.packages("lme4")
install.packages("lmerTest")
install.packages("lmtest")

In [ ]:
library(aod)      # Conducting Wald tests
library(tidyverse) # Various data management functions
library(ggplot2)  # Graphs
library(lme4)     # Estimating multilevel models
library(lmerTest) # Printing p-values for the regression coefficients
library(lmtest)   # Conducting likelihood ratio tests

In [ ]:
options(scipen=999) # disable scientific notation

### Import data

In [ ]:
df <- read.csv("https://raw.githubusercontent.com/DiarmuidM/sgsss-modeling-trajectories-2024/main/data/la-stats-1971-2021-analysis.csv", header=TRUE, na="NA", sep=",")
head(df, n=12) # view the first twelve observations

In [ ]:
str(df)

In [ ]:
df$pph_cat <- factor(df$pph_cat, levels = c(1,2,3,4,5), labels = c("Most Urban", "Very Urban", "Urban", "Rural", "Most Rural"))

In [ ]:
df$region <- factor(df$region, levels = c(1,2,3,4,5,6,7,8,9,10), labels = c("North East", "North West", "Yorkshire and The Humber", 
                                                                            "East Midlands", "West Midlands", "East of England",
                                                                           "London", "South East", "South West", "Wales"))

### Data Structure and Descriptives

#### Data structure

In [ ]:
length(unique(df$la_id))

In [ ]:
length(unique(df$year))

In [ ]:
df %>% count(year)

We have longitudinal data on 330 local authorities in England and Wales over the period 1971-2021. Each local authority is observed at the same time point and for a total of six times (the census years). Therefore we have perfectly balanced data.

Data need to be in long format for estimating growth-curve models. That is, an observation is uniquely identified by a combination of unit id (e.g., local authority) and time period (e.g., census period). An alternative data structure is the wide format, where an observation is uniquely identified by the unit id and the observations at each time period are stored in separate variables (e.g., income2021, income2022, etc).

#### Descriptives

We have one dependent / outcome variable and five independent variables / predictors / covariates:
* `charpop` - number of charities per 1000 residents in a local authority
* `town` - composite score which measures the level of material deprivation in a local authority; where a value of zero represents a local authority with a mean level of material deprivation, above zero identifies more deprived areas relative to mean deprivation, below zero identifies more affluent areas relative to mean deprivation
* `qual` - the percentage of individuals with the highest level of qualification (i.e., degree) in a local authority
* `nonbw` - the percentage of non-White individuals (non-British Isles born in 1971 and 1981, and Non-White ethnicity 1991 onwards) in a local authority
* `region` - the International Territorial Levels (ITLs) classification of the region a local authority is in
* `pph_cat` - the degree of rurality in a local authority, operationalised as an ordinal categorical measure of the number of persons per hectare 

In [ ]:
summary(df$charpop)

In [ ]:
summary(df$town)

In [ ]:
summary(df$qual)

In [ ]:
summary(df$nonbw)

In [ ]:
table(df$region)

In [ ]:
table(df$pph_cat)

List a small subset of the data:

In [ ]:
df[df$la_id == 258 | df$la_id == 91, 
       c("la_name" , "year", "charpop" , "town" , "qual" , "nonbw", "region", "pph_cat")]

**TASK:** Select a different set of local authority ids and display their observations like above.

In [ ]:
# INSERT CODE HERE #

#### Examine variation between and within local authorities

In addition to the usual exploratory activities (e.g., summarising variables, checking valid values and observations), we want to establish where there is sufficient variation in the outcome **between** and **within** units of analysis. If there isn't then we would prefer a simpler model that averaged values across units and over time i.e., a single intercept and slope for all units of analysis.

##### Between variation

In [ ]:
df_means <- df %>% group_by(la_id) %>% summarise(charpop=mean(charpop))

In [ ]:
mn_charpop <- mean(df_means$charpop)

In [ ]:
ggplot(df_means, aes(x = la_id, y = charpop)) + 
    geom_point() +
    geom_hline(yintercept=mn_charpop) + 
    theme_classic()

Let's unpack the above code and graph. We are interested in whether local authorities are sufficiently different than each other with respect to their average levels of charity density. The average across all local authorities is ~2.72, therefore we want to see how individual local authorities deviate around this value. The graph suggests that there is a moderate / large amount of **between variation** in the data.

**QUESTION:** Can you describe what the `df_means <- df %>% group_by(la_id) %>% summarise(charpop=mean(charpop))` code is doing?

**TASK:** Replace `df_means` with `df` in the `ggplot` code above - what happens and why?

In [ ]:
# INSERT CODE HERE #

##### Within variation

First let's look at a small sample of local authorities to see if theie levels of charity density change over time:

In [ ]:
df$sample <- df$la_id %in% c(258, 91, 4, 329, 155, 33)

In [ ]:
ggplot(df[df$sample == TRUE, ], aes(x = year, y = charpop)) +
    geom_point() +
    geom_line() +
    facet_wrap(~la_id) +
    theme_classic()

**QUESTION:** Do you notice any patterns in the temporal trends?

Now let's produce the temporal trends for all local authorities in the data:

In [ ]:
ggplot(df, aes(x = year, y = charpop, group = la_id)) +
    geom_point() +
    geom_line() +
    theme_classic()

**QUESTION:** How would you summarise the nature and degree of within variation in charity density?

### Estimate a random intercept (null) model

The descriptive analysis suggests that the local authority means do vary around the overall mean. Therefore we should estimate a model that allows us to capture this variation: a random intercept model. This does not include any covariates but it does allow every local authority to have its own intercept, which is likely to provide a better fit to the data than just having a single intercept for the whole sample. Thankfully we can test whether it is a better fit as follows:

#### Estimate a single-level model

In [ ]:
m1 <- lm(charpop ~ 1, data = df)
summary(m1)

Notice how the model estimates a single intercept that is approximately the mean value of charity density for the whole sample.

In [ ]:
logLik(m1) # log-likelihood of the model - used in comparing nested models

In [ ]:
-2*logLik(m1) # deviance of the model - used in comparing nested models

#### Estimate a multi-level model

In [ ]:
m2 <- lmer(charpop ~ 1 + (1 | la_id), data = df, REML = FALSE)
summary(m2)

We focus on the results reported under *Random effects* and *Fixed effects*. The latter reports the grand or overall mean of charity density for the whole sample. The former reports the variance around this overall mean by local authority - these figures are tricky to interpret so let's produce some derived findings that are more useful. 

##### Calculate the VPC/ICC

In [ ]:
rpm2 <- as.data.frame(VarCorr(m2))

# VPC/ICC = var(u)/[var(u) +var(e)]
rpm2$vcov[rpm2$grp == "la_id"] / sum(rpm2$vcov)

The VPC / ICC is a measure of the importance of the group / nested structure of the data. Our figure of ~.81 implies that over 80 per cent of the variance in the outcome is accounted for by differences **between** local authorities (and the reverse is true: ~20 per cent of the variance in the outcome is accounted for by changes over time **within** local authorities).

This coheres with our descriptive analysis, where it appears that the vast majority of local authorities have the same temporal trend (i.e., trajectory) but have considerably different mean levels of charity density. Therefore if we want to explain as much variance as possible we should focus on covariates that capture differences between local authorities.

Let's test whether the random intercept model is a better fit than a simple linear (single-level) model:

In [ ]:
lrtest(m1, m2)

Here we are looking at whether the Chisq value is statistically significant (i.e., is the reduction in log-likelihood between the models large enough to care about from a statistical perspective). In this case it is and therefore we should prefer the multi-level model.

### Estimate a random coefficent / slope model

While it is important to capture the **between variation** in our models, we should not lose sight of the fact that our substantive interest is in estimating trajectories over time. Therefore we need to incorporate our measure of time / period into the next set of models, as well as checking if the effect of time (or the shape of the trajectory) varies by local authority.

In [ ]:
df$period2 <- df$period^2

**QUESTION:** Why are we creating a quadratic / squared term for our time variable?

In [ ]:
m3 <- lmer(charpop ~ period + period2 + (1 | la_id), data = df, REML = FALSE)
summary(m3)

Let's focus on the fixed part of the model:
* For every increase in census period, charity density increases linearly by .86 on average
* For every increase in census period, the linear increase in charity density decreases by -.12 on average

Therefore there is a concave / quadratic trajectory over time: charity density increases linearly initially but eventually becomes n-shaped. We saw that in the descriptives where charity density increased rapidly between 1971 and 2001, before tailing off / plateauing until 2021.

Is there evidence of variation in the shape of this trajectory by local authority:

In [ ]:
m4 <- lmer(charpop ~ period + period2 + (1 + period + period2 | la_id), data = df, REML = FALSE)
summary(m4)

Looking at the results unde *Random effects*, it appears that the linear change over time does vary by local authority (i.e., some areas have faster / slower rates of change), but the quadratic change does not vary (i.e., local authorities experience essentially the same slowdown in linear growth). Therefore we could take out the `period2` covariate from the random part of the regression equation and model fit would be unaffected.

In [ ]:
m4 <- lmer(charpop ~ period + period2 + (1 + period | la_id), data = df, REML = FALSE)
summary(m4)

We can interpret another useful result from this model: the correlation between the random intercept and random slope. It is calculated as .25, which implies a small, positive correlation. We can interpret this as follows: local authorities that have higher than average levels of charity density have higher than average rates of change. Put more simply, the authorities with the most charities grew fastest.

In [ ]:
lrtest(m2, m4)

**QUESTION:** Should we prefer model 4 (random slope) over model 2 (random intercept)?

#### Predict and plot trajectories

Let's take our first look at the model-implied trajectories for the local authorities:

In [ ]:
df$xbu1 <- predict(m4)

# Plot the fitted growth curves
ggplot(df, aes(x = year, y = xbu1, group = la_id)) +
    geom_line() +
    theme_classic()

**QUESTION:** Describe the shape of these trajectories, focus particularly on the variation in intercepts and slopes.

### Estimate a random coefficient / slope model with time-invariant covariates

Our latest model does a good job of estimating the trajectories seen in the descriptive analysis, and thus provide decent predictions for the level of charity density in a local authority at a given census period. We can aim to reduce the unexplained variance, especially the differences between local authorities, by including more covariates. We will focus on time-invariant covariates in the first instance: that is, variables whose values are constant within but different across local authorities (e.g., region).

In [ ]:
df$region <- relevel(df$region, "London") # set London as the base / reference category for the region variable for model estimation

In [ ]:
m5 <- lmer(charpop ~ period + period2 + region + (1 + period | la_id), data = df, REML = FALSE)
summary(m5)

**TASK:** Conduct a likelihood ratio test of whether model 5 is a better fit for the data than model 4. Also look at the variance of the intercept in model 4 and 5 and see which one is lower (Hint: look in the *Random effects* section).

In [ ]:
# INSERT CODE HERE #

**TASK:** Plot the predicted trajectories using model 5.

In [ ]:
# INSERT CODE HERE # 

### Estimate a random coefficient / slope model with time-invariant and time-varying covariates

You may have noticed that including time-invariant covariates reduced the unexplained variance between local authorities but not for within local authorities. This intuitively makes sense: a covariate that measures differences between units will not help explain differences within units. So let's address the latter by including variables that change over time e.g., material deprivation, ethnic composition.

In [ ]:
df$pph_cat <- relevel(df$pph_cat, "Rural") # set Rural as the base / reference category for the urban/rural variable for model estimation

In [ ]:
m6 <- lmer(charpop ~ period + period2 + region + pph_cat + town + qual + nonbw + (1 + period | la_id), data = df, REML = FALSE)
summary(m6)

**QUESTION:** Interpret the results from the fixed part of the model. For example, what is the expected change in charity density for a one-unit increase in material deprivation (`town`)? Do rural areas have higher density than urban areas?

**QUESTION:** Look at the variances for the intercept and slope in the *Random effects* section - have they decreased? How can the variance for the intercept decrease by adding time-varying covariates?

In [ ]:
lrtest(m5, m6)

In [ ]:
rpm6 <- as.data.frame(VarCorr(m6))

# VPC/ICC = var(u)/[var(u) +var(e)]
rpm6$vcov[rpm6$grp == "la_id"] / sum(rpm6$vcov)
# note we are only interested in first result

### Use final model to generate estimates of the random intercepts and coefficients / slopes

Now we have settled on a final model that fits the data well, let's produce some derived results to aid our interpretation of the findings. In particular, we will estimate the intercepts and slopes for each local authority and cehck whether there are any particularly odd / outlier areas.

#### Predict and plot trajectories

In [ ]:
df$xbu1 <- predict(m6)

# Plot the fitted growth curves
ggplot(df, aes(x = year, y = xbu1, group = la_id)) +
    geom_line() +
    theme_classic()

We can also plot the fitted trajectories by interesting variables (including those not in the model) in our data:

In [ ]:
ggplot(df, aes(x = year, y = xbu1, group = la_id)) +
    geom_line() +
    facet_wrap(~region) +
    theme_classic()

**TASK:** Plot the predicted trajectories by the `pph_cat` variable. What do you notice?

#### Produce random effects for each local authority

In [ ]:
u01m6 <- data.frame(ranef(m6), condVar = TRUE)
sample_n(u01m6, 20) # sample 20 observations from the random effects dataset

For the subsequent tasks it is easier to split this data into two: one for the random intercepts and one for the random coeffients / slopes:

In [ ]:
u0m6 <- u01m6[u01m6$term=="(Intercept)",]
u1m6 <- u01m6[u01m6$term=="period",]

We can now produce a 'caterpillar plot' of the ranking of each set of random effects - this will allow us to detect outliers i.e., local authorities that are statistically significantly above or below average.

In [ ]:
# Calculate the lower and upper values of the confidence intervals
u0m6$lower <- u0m6$condval - 1.96*u0m6$condsd
u0m6$upper <- u0m6$condval + 1.96*u0m6$condsd
# By multiplying  the standard errors by 1.96 we calculate 95% confidence
# intervals

# Rank the local authority effects
u0m6$rank <- rank(u0m6$condval)
sample_n(u0m6, 10)

The `rank` variable now captures the ranking of the random intercepts, from lowest to highest.

In [ ]:
ggplot(u0m6, aes(x = rank, y = condval, ymin = lower, ymax = upper)) +
    geom_hline(yintercept = 0) +
    geom_errorbar() +
    geom_point() +
    theme_classic()

Let's unpack this as there's a lot going on:
* The horizontal line represents the intercept for the whole sample (i.e., the mean level of charity density in 1971). It is centred on 0 so we can better identify deviations from the average. 
* The dots and error bars represent the deviation of each specific local authority from the overall intercept. If the bars **do not** overlap with 0 then that is evidence of statistically significant deviations from the average (i.e., some local authorities have much higher or lower levels of charity density in 1971 than expected).
* The fact that there are obvious deviations from the average suggests that there are covariates missing from the model that might explain differences between local authorities. (Remember that random effects are residuals - the difference between observed values and values predicted by your model. Ideally these residuals are unexplainable e.g., due to luck, random chance etc, but this is only the case when you have **only** and **all** relevant covariates in your model (rarely the case)).

Let's produce the same plot but for the random coefficients / slopes:

In [ ]:
# Calculate the lower and upper values of the confidence intervals
u1m6$lower <- u1m6$condval - 1.96*u1m6$condsd
u1m6$upper <- u1m6$condval + 1.96*u1m6$condsd
# By multiplying  the standard errors by 1.96 we calculate 95% confidence
# intervals

# Rank the local authority effects
u1m6$rank <- rank(u1m6$condval)
sample_n(u1m6, 10)

In [ ]:
ggplot(u1m6, aes(x = rank, y = condval, ymin = lower, ymax = upper)) +
    geom_hline(yintercept = 0) +
    geom_errorbar() +
    geom_point() +
    theme_classic()

We can see that the random coefficients / slopes do not deviate from the overall average by much: there are only a handful of local authorities that have faster rates of change (steeper linear trajectories) than expected.

## Conclusion

We have covered a great deal of the essential elements of modeling trajectories in the social sciences (well done). However there are plenty of other considerations to be made as you engage more deeply with this topic:
* **Cross-level interactions** - including interactions between variables measured at different levels (e.g., `region` and `town` in our example).
* **Estimating and examining variance functions** - is the unexplained variance at the higher level (e.g., local authority) growing or shrinking over time? That is, is it becoming more or less difficult to predict the outcome over time?
* **Specifying different error structures** - perhaps it makes sense to allow the VPC / ICC to vary over time (see point above), or to allow the residual variance of the intercept to depend on categories of an important covariate (e.g., maybe regions have different levels of unexplained variance?).
* **Autocorrelation** - perhaps a model that allows the residuals to be autocorrelated (i.e., dependent on past values) is a better fit than a growth curve / trajectory?

You are encouraged to see the suggested readings for resources you can use to deepen your understanding of and fluency in this topic.

--END OF FILE--